#**ChatBot**

#(Retrieval Based)





---





###**Importing Necessary Libraries**


In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


###**Loading and Pre-Processing Data**

In [3]:
data = open('intents_restaurant.json').read()
intents = json.loads(data)

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
words=[]
classes = []
documents = []

ignore_words = ['?', '!']


---
Tokenize Each Word




Add Documents in the Corpus

Add Classes

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


---
Lemmaztize and lower each word and remove duplicates

In [9]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

---
Sort classes

In [10]:
classes = sorted(list(set(classes)))

---
Documents = combination between patterns and intents

In [11]:
print (len(documents), "documents")

42 documents


---

Classes = intents

In [12]:
print (len(classes), "classes", classes)

8 classes ['food_preferences', 'food_preferences_search', 'foodtruck_search', 'goodbye', 'greeting', 'options', 'restaurant_search', 'thanks']


---
Words = all words, vocabulary

In [13]:
print (len(words), "unique lemmatized words", words)

77 unique lemmatized words ["'s", ',', 'a', 'anyone', 'are', 'awesome', 'be', 'book', 'by', 'bye', 'can', 'chatting', 'could', 'customer', 'data', 'day', 'detail', 'do', 'entry', 'find', 'food', 'foodtruck', 'foodtrucks', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'preference', 'provide', 'related', 'restaurant', 'result', 'search', 'searching', 'see', 'show', 'support', 'table', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'up', 'want', 'what', 'you']


.

.


.
Saving the words and classes (using pickle)

In [14]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

###**Creating Training and Testing Data**

In [15]:
training = []

# create an empty array for output
output_empty = [0] * len(classes)

Bag of words for every sentence

In [16]:
for doc in documents:
    # initialize our bag of words
    bag = []

    # list of tokenized words for the pattern
    pattern_words = doc[0]

    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

Shuffle Data and Convert to np.array

In [17]:
random.shuffle(training)
training = np.array(training)

# Train and Test lists. 
# X : patterns, Y : intents

train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


###**Building Model**

Model 

Layer 1 : 128 Neurons

Layer 2 : 64 Neurons 

Layer 3 : Number of Neurons = Number of Intents (Predict output intent for each using softmax)

In [18]:
model = Sequential()

model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(train_y[0]), activation='softmax'))

###**Compile and Fit Model**

Using

**Stochastic gradient descent with Nesterov accelerated gradient** gives high performance 

In [19]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [20]:
mod = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
42/42 [==============================] - 0s 8ms/step - loss: 2.0809 - accuracy: 0.1905
Epoch 2/200
42/42 [==============================] - 0s 561us/step - loss: 2.0600 - accuracy: 0.1190
Epoch 3/200
42/42 [==============================] - 0s 420us/step - loss: 1.9968 - accuracy: 0.1905
Epoch 4/200
42/42 [==============================] - 0s 567us/step - loss: 1.9023 - accuracy: 0.4762
Epoch 5/200
42/42 [==============================] - 0s 475us/step - loss: 1.8593 - accuracy: 0.4286
Epoch 6/200
42/42 [==============================] - 0s 448us/step - loss: 1.7731 - accuracy: 0.4762
Epoch 7/200
42/42 [==============================] - 0s 459us/step - loss: 1.6635 - accuracy: 0.4762
Epoch 8/200
42/42 [==============================] - 0s 532us/step - loss: 1.4834 - accuracy: 0.5476
Epoch 9/200
42/42 [==============================] - 0s 482us/step - loss: 1.3056 - accuracy: 0.5000
Epoch 10/200
42/42 [==============================] - 0s 493us/step - loss: 1.2940 - accuracy

###**Save Model**

In [21]:
model.save('chatbot_final_model.h5', mod)
print("model created")

model created
